In [1]:
from __future__ import annotations

from flamekit.datasets import SEMDataset
from flamekit.io_fronts import Case
from mpi4py import MPI
from pathlib import Path
import pandas as pd

# INPUT PARAMETERS (CAPITALS)
# ============================================================

TIME_STEP = 335
MULTI_TIME_STEP = True

TIME_STEP_START = 270
TIME_STEP_END = 301

PHI = 0.40
LAT_SIZE = "025"
T_REF = 300
P_REF = 1e05

POST = True

DATA_BASE_DIR = Path("data")
OUTPUT_BASE_DIR = Path("../isocontours")  # keep using your unified folder convention

FILE_NAME = "po_postPremix" if POST else "premix"

SCALARS = [
    "H2", "O2", "H2O", "H", "O", "OH", "HO2", "H2O2", "N2", "HRR",
    "curvature", "stretch_rate", "DW_FDS",
    "abs_flame_prop_vel_normal",
    "flow_velocity_normal", "flow_velocity_tangential",
    "strain_rate", "tangential_strain_rate", "normal_strain_rate",
    "density_ratio_sigma", "gradT", "gradT_normal",
    "total_heat_conduction", "heat_conduction_normal", "heat_conduction_tangential",
    "FDS_src_term", "FDS_diffusion_term", "FDS_diff_velocity_term",
    "H2_diffusion_total", "O2_diffusion_total", "H_diffusion_total",
    "vorticity",
]

CANTERA_INPUTS = ["../chemical_mech/BurkeH2.yaml", None, T_REF, P_REF]


# PATH HELPERS (Case-style, but for FULL FIELD)
# - Uses your existing folder naming convention (h400x{lat}_ref[_Soret...])
# - Output filename: extracted_field[_post]_{timestep}.csv   (NO iso_...)
# ============================================================

def _suffix_from_case(case: Case) -> str:
    # mirrors your earlier convention
    suffix = f"h400x{case.lat_size}_ref"
    if getattr(case, "soret_term", False) and getattr(case, "soret_positive", False):
        suffix = f"h400x{case.lat_size}_ref_Soret_positive"
    elif getattr(case, "soret_term", False):
        suffix = f"h400x{case.lat_size}_ref_Soret"
    return suffix


def _data_folder(case: Case, data_base_dir: Path) -> Path:
    folder = data_base_dir / f"phi{case.phi:.2f}" / _suffix_from_case(case)
    if getattr(case, "multiple_runs", False):
        folder = folder / f"RUN0{getattr(case, 'n_run', 0)}"
    return folder


def _output_folder(case: Case, output_base_dir: Path) -> Path:
    out = output_base_dir / f"phi{case.phi:.2f}" / _suffix_from_case(case)
    if getattr(case, "multiple_runs", False):
        out = out / f"RUN0{getattr(case, 'n_run', 0)}"
    return out


def _field_filename(case: Case) -> str:
    # NO iso_...
    if getattr(case, "post", False):
        return f"extracted_field_post_{case.time_step}.csv"
    return f"extracted_field_{case.time_step}.csv"


def _field_out_path(case: Case, output_base_dir: Path) -> Path:
    return _output_folder(case, output_base_dir) / _field_filename(case)


# RUN
# ============================================================

comm = MPI.COMM_WORLD
rank = comm.rank


def _process_one_timestep(ts: int):
    case = Case(
        base_dir=OUTPUT_BASE_DIR,  # kept for consistency; not used directly here
        phi=PHI,
        lat_size=LAT_SIZE,
        time_step=ts,
        post=POST,
    )

    folder_name = _data_folder(case, DATA_BASE_DIR)
    save_dir = _output_folder(case, OUTPUT_BASE_DIR)
    out_path = _field_out_path(case, OUTPUT_BASE_DIR)

    if rank == 0:
        save_dir.mkdir(parents=True, exist_ok=True)

    ds = SEMDataset(
        file_name=FILE_NAME,
        folder_name=str(folder_name),
        time_step=case.time_step,
        comm=comm,
        scalar_names=SCALARS,
    )

    df = ds.create_dataframe(
        compute_vel_jacobian=False,
        compute_vel_hessian=False,
        compute_reaction_rates=True,
        cantera_inputs=CANTERA_INPUTS,
    )

    # Save only on rank 0
    if rank == 0:
        df.to_csv(out_path, index=False)
        print(f"[rank0] Saved full field: {out_path}  (n={len(df)})")

    # ensure memory release
    del df


if MULTI_TIME_STEP:
    for ts in range(TIME_STEP_START, TIME_STEP_END + 1):
        if rank == 0:
            print(f"\n=== Processing TIME_STEP={ts} ===")
        _process_one_timestep(ts)
else:
    if rank == 0:
        print(f"\n=== Processing TIME_STEP={TIME_STEP} ===")
    _process_one_timestep(TIME_STEP)



=== Processing TIME_STEP=270 ===
2025-12-24 12:42:35,508 - Mesh - INFO - Initializing empty Mesh object.
2025-12-24 12:42:35,509 - Field - INFO - Initializing empty Field object
2025-12-24 12:42:35,509 - pynekread - INFO - Reading file: ../data\phi0.40\h400x025_ref/po_postPremix0.f00001
2025-12-24 12:42:35,541 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-12-24 12:42:35,542 - Mesh - INFO - Initializing common attributes.
2025-12-24 12:42:35,543 - Mesh - INFO - Getting vertices
2025-12-24 12:42:35,545 - Mesh - INFO - Getting edge centers
2025-12-24 12:42:35,572 - Mesh - INFO - Facet centers not available for 2D
2025-12-24 12:42:35,572 - Mesh - INFO - Mesh object initialized.
2025-12-24 12:42:35,573 - Mesh - INFO - Mesh data is of type: float64
2025-12-24 12:42:35,574 - Mesh - INFO - Elapsed time: 0.03290490002837032s
2025-12-24 12:42:35,574 - pynekread - INFO - Reading field data
2025-12-24 12:42:35,574 - pynekread - INFO - Overwriting fld object
2025-12-24 12:42:3